In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_predict

from gensim.test.utils import common_texts
from gensim.models import Word2Vec
model = Word2Vec(sentences=common_texts, vector_size=100, window=5, min_count=1, workers=4)

In [8]:
import pandas as pd
import numpy as np
import jieba

data_dir = 'https://mirror.coggle.club/dataset/coggle-competition/'
train_data = pd.read_csv(data_dir + 'intent-classify/train.csv', sep='\t', header=None)
test_data = pd.read_csv(data_dir + 'intent-classify/test.csv', sep='\t', header=None)
train_data[0] = train_data[0].apply(jieba.lcut)
test_data[0] = test_data[0].apply(jieba.lcut)

In [9]:
model = Word2Vec(
    sentences=list(train_data[0].values[:]) + list(test_data[0].values[:]), 
vector_size=30, window=5, min_count=1, workers=4)

In [10]:
model.wv.most_similar('打开')

[('热水器', 0.98893141746521),
 ('关掉', 0.9859920740127563),
 ('加湿器', 0.9854778051376343),
 ('这时候', 0.9815304279327393),
 ('电脑', 0.9793143272399902),
 ('，', 0.9790096879005432),
 ('蒸', 0.975031852722168),
 ('洗衣机', 0.9734700918197632),
 ('准备', 0.9727935791015625),
 ('把', 0.9721974730491638)]

In [11]:
train_w2v = train_data[0].apply(lambda x: model.wv[x].mean(0))
test_w2v = test_data[0].apply(lambda x: model.wv[x].mean(0))

train_w2v = np.vstack(train_w2v)
test_w2v = np.vstack(test_w2v)

In [12]:
cv_pred = cross_val_predict(
    LinearSVC(),
    train_w2v, train_data[1]
)
print(classification_report(train_data[1], cv_pred))

/home/lyz/anaconda3/envs/py311/lib/python3.11/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/home/lyz/anaconda3/envs/py311/lib/python3.11/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/lyz/anaconda3/envs/py311/lib/python3.11/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/home/lyz/anaconda3/envs/py311/lib/python3.11/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/lyz/anaconda3/envs/py311/lib/python3.11/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default val

                       precision    recall  f1-score   support

         Alarm-Update       0.91      0.93      0.92      1264
           Audio-Play       0.00      0.00      0.00       226
       Calendar-Query       0.93      0.97      0.95      1214
        FilmTele-Play       0.56      0.63      0.59      1355
HomeAppliance-Control       0.84      0.92      0.88      1215
           Music-Play       0.73      0.78      0.75      1304
                Other       0.10      0.01      0.02       214
         Radio-Listen       0.85      0.84      0.85      1285
       TVProgram-Play       0.69      0.05      0.09       240
         Travel-Query       0.84      0.92      0.88      1220
           Video-Play       0.70      0.73      0.71      1334
        Weather-Query       0.82      0.83      0.83      1229

             accuracy                           0.79     12100
            macro avg       0.66      0.63      0.62     12100
         weighted avg       0.76      0.79      0.77 